In [83]:
import os
from pinecone import Pinecone, ServerlessSpec
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()

True

In [84]:
pc = Pinecone(api_key = os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=768, metric="cosine", spec = ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '295e592f54cc9795823da04fd7607117', 'Date': 'Tue, 03 Sep 2024 01:16:06 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [85]:
import json
data = json.load(open("reviews.json"))
data['reviews']


[{'professor': 'John Doe',
  'subject': 'Introduction to Computer Science',
  'stars': 4,
  'review': 'Professor Doe was a great teacher. He explained concepts clearly and was always willing to answer questions.'},
 {'professor': 'Jane Smith',
  'subject': 'Calculus I',
  'stars': 3,
  'review': 'The course was challenging, but Professor Smith was a good lecturer. She provided helpful examples and practice problems.'},
 {'professor': 'David Lee',
  'subject': 'English Composition',
  'stars': 5,
  'review': 'Professor Lee is an amazing teacher. He is passionate about literature and inspired me to love reading.'},
 {'professor': 'Emily Chen',
  'subject': 'Biology I',
  'stars': 2,
  'review': "The course material was too much for me. Professor Chen's lectures were difficult to follow."},
 {'professor': 'Thomas Brown',
  'subject': 'History of the United States',
  'stars': 4,
  'review': 'Professor Brown is a knowledgeable historian. He made the course interesting and engaging.'},
 {'p

In [86]:
processed_data = []
genai.configure(api_key = os.getenv('GEMINI_API_KEY'))
for review in data['reviews']:
    result = genai.embed_content(
    model="models/text-embedding-004", content=review['review'], output_dimensionality=768
    )
    embedding = result["embedding"]
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [87]:
processed_data[0]

{'values': [0.050226025,
  -0.016519066,
  -0.029485252,
  0.058657706,
  0.009006167,
  0.01649257,
  -0.014465294,
  0.006812316,
  0.03741776,
  0.0031112882,
  0.055169396,
  0.03632456,
  0.039584696,
  -0.022995207,
  0.03485217,
  -0.0394114,
  -0.050925873,
  0.012741427,
  -0.05675219,
  0.018911418,
  0.017818948,
  -0.014567799,
  0.010138968,
  -0.055859853,
  0.011489537,
  -0.009371707,
  0.00080662465,
  -0.064000376,
  0.028218055,
  -0.031696975,
  0.019722115,
  0.017259471,
  -0.06384553,
  -0.004148659,
  0.0019741417,
  0.0686884,
  -0.014594898,
  -0.032215092,
  0.051493227,
  -0.029613337,
  -0.017717315,
  -0.05468274,
  0.0018730811,
  0.015883295,
  -0.037903253,
  0.0057794834,
  -0.019374723,
  0.10221813,
  0.033029653,
  0.09743941,
  -0.011646162,
  0.03922772,
  -0.05181747,
  0.003249802,
  0.028080523,
  -0.031230435,
  -0.02837782,
  -0.02291459,
  0.033760726,
  0.015450839,
  0.0025976377,
  0.013102066,
  -0.045927715,
  -0.0847376,
  0.031833652,

In [88]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}